In [1]:
input_width = 160
input_height = 100
channels = 3
class_number = 12

In [2]:
from keras.models import load_model
import keras.backend as K
def actual_acc(y_true, y_pred):
    return K.equal(K.argmax(y_pred), K.argmax(y_true))

model = load_model('models/worm_single_large_adam.h5',custom_objects={'actual_acc':actual_acc})
model.summary()

Using TensorFlow backend.
W0128 01:59:33.583799 10996 deprecation_wrapper.py:119] From Q:\Program Files\Anaconda3\lib\site-packages\keras-2.2.4-py3.6.egg\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0128 01:59:33.844802 10996 deprecation_wrapper.py:119] From Q:\Program Files\Anaconda3\lib\site-packages\keras-2.2.4-py3.6.egg\keras\backend\tensorflow_backend.py:529: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0128 01:59:33.924449 10996 deprecation_wrapper.py:119] From Q:\Program Files\Anaconda3\lib\site-packages\keras-2.2.4-py3.6.egg\keras\backend\tensorflow_backend.py:4420: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0128 01:59:34.039455 10996 deprecation_wrapper.py:119] From Q:\Program Files\Anaconda3\lib\site-packages\keras-2.2.4-py3.6.egg\keras\backend\tensorflow_backend.py:4255: The name tf.nn.max_pool is depreca

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 158, 32)       896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 79, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 77, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 38, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 36, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 18, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 16, 128)       

In [6]:
from screen_consts import WIDTH, HEIGHT, get_coordinates_from_direction
from grabscreen import grab_screen
from image_preproc import preproc_img
from getkeys import key_check
from image_preproc import prepare_image
import win32api, win32con
import time
import numpy as np
import cv2

def mouse_down(x,y):
    win32api.SetCursorPos((x,y))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN, x, y, 0, 0)

def mouse_up(x, y):
    win32api.SetCursorPos((x, y))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP, x, y, 0, 0)

    
from PIL import ImageGrab
def get_image():
    img = np.array(ImageGrab.grab())
    img = preproc_img(img)
    return img

def detect_lose(img): # input RGB
    img = roi(img, [np.array([[WIDTH/2-12, HEIGHT/2-5], [WIDTH/2+12, HEIGHT/2-5], [WIDTH/2+12, HEIGHT/2+3], [WIDTH/2-12, HEIGHT/2+3]], np.int32)])
    b = np.average(np.array(img[:,:,2]))
    g = np.average(np.array(img[:,:,1]))
    r = np.average(np.array(img[:,:,0]))
    return r < 0.6 and g > 1.5 and b > 0.5

class_number = 12
img = grab_screen()
scr_W, scr_H = len(img[0]), len(img)


print("Start/Pause action with Alt+T")
paused = True
prev_img = 0
img = 0
while True:
    clock_prev = time.clock()
    keys = key_check()
    if "ALT" in keys and "T" in keys:
        paused ^= 1
        if paused:
            print("Stop action, press Alt+T to continue")
        else:
            print("Start action in 3 sec.")
            for i in list(range(3))[::-1]:
                time.sleep(1)
                print(i + 1)
            print("action!")
        time.sleep(1)
        
    if not paused:
        img = get_image().reshape(-1,input_height, input_width,3)
        
        if "C" in keys or np.sum(prev_img) == np.sum(img):
            offset = -55
            mouse_up(scr_W//2, scr_H//2+offset)
            mouse_down(scr_W//2, scr_H//2+offset)
            mouse_up(scr_W//2, scr_H//2+offset)

        prev_img = img.copy()
        img = img / 255
        
        prediction_raw = model.predict(img)
        prediction = np.argmax(prediction_raw)
        x, y = get_coordinates_from_direction(prediction, scr_W, scr_H, class_number)
        mouse_up(x, y)
        mouse_down(x, y)
        
        #print(np.round(model.predict(prep_img), 2), prediction)
        #print('Took {} ms'.format(round((time.clock() - clock_prev)*1000)))

Start/Pause action with Alt+T
Start action in 3 sec.
3
2
1
action!
0 1429652
1429652 1429652
1429652 1428119
1428119 1418873
1418873 1304793
1304793 1305359
1305359 1304611
1304611 1304776
1304776 1305033
1305033 1304918
1304918 1304894
1304894 1305498
1305498 1305248
1305248 1304521
1304521 1304918
1304918 548930
548930 553215
553215 608895
608895 640984
640984 642786
642786 644145
644145 647293
647293 649960
649960 655118
655118 660298
660298 668180
668180 676926
676926 686055
686055 693236
693236 704330
704330 719974
719974 721667
721667 723630
723630 722944
722944 719248
719248 721128
721128 724819
724819 728281
728281 727278
727278 728643
728643 719282
719282 731705
731705 733400
733400 735423
735423 736044
736044 733125
733125 732226
732226 732351
732351 734575
734575 734925
734925 731620
731620 732709
732709 735548
735548 736481
736481 736338
736338 737065
737065 735978
735978 733679
733679 731151
731151 733231
733231 731233
731233 740064
740064 761015
761015 767658
767658 76934

1304793 1305412
1305412 1304849
1304849 1305124
1305124 1305080
1305080 1305498
1305498 1304857
1304857 1304912
1304912 1305100
1305100 1305229
1305229 1305556
1305556 1304512
1304512 552930
552930 643114
643114 653861
653861 647257
647257 658022
658022 661139
661139 665861
665861 670592
670592 679434
679434 683806
683806 692291
692291 703683
703683 709480
709480 738826
738826 751524
751524 758696
758696 756575
756575 753360
753360 753024
753024 752031
752031 751449
751449 749625
749625 746377
746377 747224
747224 751780
751780 752970
752970 754930
754930 759055
759055 754809
754809 750222
750222 749528
749528 740396
740396 738188
738188 735781
735781 732218
732218 729803
729803 727851
727851 728395
728395 725136
725136 726432
726432 728036
728036 724059
724059 726117
726117 726980
726980 729555
729555 730531
730531 731237
731237 731251
731251 727934
727934 725988
725988 722284
722284 720203
720203 720038
720038 718988
718988 715672
715672 716569
716569 715680
715680 713934
713934 7175

818178 816212
816212 815462
815462 815611
815611 814039
814039 812485
812485 810421
810421 807840
807840 807659
807659 806516
806516 806534
806534 806360
806360 811463
811463 812186
812186 816088
816088 754971
754971 792940
792940 824675
824675 861134
861134 895781
895781 928209
928209 964401
964401 999701
999701 1029702
1029702 1063577
1063577 1092070
1092070 1124473
1124473 1138106
1138106 1138933
1138933 1138789
1138789 1138301
1138301 1139889
1139889 1139841
1139841 1141081
1141081 1143701
1143701 1142317
1142317 1144978
1144978 1144296
1144296 1145085
1145085 1143788
1143788 1146186
1146186 1145217
1145217 1150126
1150126 1150817
1150817 1153129
1153129 1152948
1152948 1154359
1154359 1155975
1155975 1156943
1156943 1160202
1160202 1163470
1163470 1163414
1163414 1169590
1169590 1168540
1168540 1171364
1171364 1173012
1173012 1177442
1177442 1176363
1176363 1177915
1177915 1179229
1179229 1177073
1177073 1176639
1176639 1175392
1175392 1176403
1176403 1176339
1176339 1176915
11769

1532761 1527805
1527805 1523827
1523827 1509561
1509561 1492237
1492237 1480715
1480715 1469574
1469574 1457887
1457887 1435950
1435950 1403190
1403190 1359134
1359134 1314058
1314058 1280245
1280245 1268414
1268414 1284443
1284443 1304801
1304801 1334381
1334381 1359798
1359798 1380156
1380156 1389363
1389363 1386279
1386279 1384777
1384777 1397483
1397483 1416530
1416530 1456545
1456545 1492484
1492484 1534455
1534455 1574278
1574278 1610957
1610957 1641357
1641357 1677363
1677363 1710105
1710105 1750001
1750001 1777157
1777157 1806476
1806476 1831482
1831482 1860495
1860495 1889997
1889997 1937144
1937144 1991337
1991337 2051056
2051056 2095570
2095570 2137026
2137026 2134525
2134525 2130827
2130827 2119164
2119164 2111653
2111653 2107603
2107603 2114447
2114447 2122304
2122304 2141984
2141984 2162099
2162099 2182163
2182163 2192125
2192125 2194812
2194812 2184020
2184020 2167188
2167188 2151938
2151938 2138981
2138981 2140529
2140529 2155046
2155046 2184967
2184967 2217149
2217149 

752426 747919
747919 742749
742749 738558
738558 732735
732735 726898
726898 719540
719540 724780
724780 729066
729066 733843
733843 736367
736367 740715
740715 744080
744080 745843
745843 751840
751840 754761
754761 756447
756447 754126
754126 752731
752731 751636
751636 750892
750892 749065
749065 746218
746218 747500
747500 750531
750531 753412
753412 757410
757410 756786
756786 760264
760264 756716
756716 750748
750748 747746
747746 748240
748240 751682
751682 755539
755539 761536
761536 767398
767398 767540
767540 769953
769953 773379
773379 772206
772206 767086
767086 763276
763276 762963
762963 763560
763560 765232
765232 764640
764640 762911
762911 762442
762442 762085
762085 765160
765160 763165
763165 758439
758439 754198
754198 758381
758381 757323
757323 757853
757853 754040
754040 751977
751977 745622
745622 744847
744847 744059
744059 742028
742028 742325
742325 738260
738260 736851
736851 733472
733472 730521
730521 727826
727826 726990
726990 724308
724308 724412
724412

...

1078237 1099678
1099678 1090265
1090265 1092215
1092215 1084889
1084889 1086581
1086581 1094554
1094554 1104459
1104459 1120964
1120964 1142916
1142916 1157231
1157231 1160253
1160253 1142482
1142482 1123207
1123207 1096325
1096325 1072511
1072511 1053534
1053534 1060465
1060465 1064633
1064633 1070376
1070376 1065411
1065411 1046261
1046261 1016604
1016604 980882
980882 945745
945745 929761
929761 930132
930132 932865
932865 944768
944768 947456
947456 917695
917695 886494
886494 851318
851318 831454
831454 820746
820746 816207
816207 821581
821581 824377
824377 838879
838879 838697
838697 833148
833148 822463
822463 819982
819982 807537
807537 804159
804159 805053
805053 790164
790164 792888
792888 791294
791294 796391
796391 795794
795794 796958
796958 791078
791078 798432
798432 802569
802569 805076
805076 806376
806376 806319
806319 806777
806777 805839
805839 810441
810441 808014
808014 810115
810115 807136
807136 808721
808721 807848
807848 811113
811113 810575
810575 815010
815

951924 942589
942589 921386
921386 929904
929904 931917
931917 926032
926032 924413
924413 923962
923962 916796
916796 918652
918652 921374
921374 928182
928182 931466
931466 927898
927898 932201
932201 942851
942851 947856
947856 958080
958080 964331
964331 967380
967380 974410
974410 982506
982506 994545
994545 998389
998389 1008307
1008307 1058101
1058101 1066052
1066052 1070051
1070051 1063932
1063932 1030494
1030494 1012962
1012962 996298
996298 980371
980371 999440
999440 1009872
1009872 1015416
1015416 1022774
1022774 1031663
1031663 1049922
1049922 1059895
1059895 1076492
1076492 1080163
1080163 1085264
1085264 1071774
1071774 1071066
1071066 1072851
1072851 1081883
1081883 1087398
1087398 1086197
1086197 1084004
1084004 1024542
1024542 1017813
1017813 1015577
1015577 1017583
1017583 1016927
1016927 1012914
1012914 1016060
1016060 1013673
1013673 1022334
1022334 1021051
1021051 1031838
1031838 1034375
1034375 1036809
1036809 1030841
1030841 1033264
1033264 1029742
1029742 10348

697327 696348
696348 692093
692093 703325
703325 702428
702428 700699
700699 690775
690775 689600
689600 695441
695441 695992
695992 696018
696018 694420
694420 694855
694855 695866
695866 688041
688041 689103
689103 687647
687647 689592
689592 698176
698176 702590
702590 709217
709217 709511
709511 698944
698944 704491
704491 704515
704515 695467
695467 706135
706135 703818
703818 701973
701973 701353
701353 698347
698347 699230
699230 698233
698233 693098
693098 692587
692587 694191
694191 693188
693188 691320
691320 691735
691735 690449
690449 693361
693361 696052
696052 694796
694796 698007
698007 698366
698366 700989
700989 701687
701687 704336
704336 705786
705786 704750
704750 704181
704181 704327
704327 703015
703015 702037
702037 705454
705454 706116
706116 708719
708719 709526
709526 710237
710237 712233
712233 712172
712172 702115
702115 711711
711711 710129
710129 710857
710857 711461
711461 711468
711468 709007
709007 709958
709958 708598
708598 702605
702605 696184
696184

675010 684270
684270 697759
697759 721880
721880 731226
731226 753702
753702 769028
Stop action, press Alt+T to continue


KeyboardInterrupt: 

# VIEW INPUT

In [ ]:
%matplotlib notebook
import cv2
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

def grab_frame():
    return get_image()

#create two subplots
gridsize = (2, 1)
fig = plt.figure(figsize=(6, 8))
ax1 = plt.subplot2grid(gridsize, (0, 0))

#create two image plots
im1 = ax1.imshow(grab_frame(), cmap='Greys_r')

def update(i):    
    im1.set_data(grab_frame())

ani = FuncAnimation(plt.gcf(), update, interval=10)
plt.show()